In [ ]:
CREATE TYPE bool_sigla AS ENUM {'t', 'f'}

In [ ]:
%%sql
DROP TABLE IF EXISTS Listings CASCADE;
CREATE TABLE airbnb_listings (
    id INT PRIMARY KEY,
    listing_url TEXT,
    scrape_id BIGINT,
    last_scraped DATE,
    source TEXT,
    name TEXT,
    description TEXT,
    neighborhood_overview TEXT,
    picture_url TEXT,
    host_id INT,
    host_url TEXT,
    host_name TEXT,
    host_since DATE,
    host_location TEXT,
    host_about TEXT,
    host_response_time TEXT,
    host_response_rate TEXT,
    host_acceptance_rate TEXT,
    host_is_superhost bool_sigla,
    host_thumbnail_url TEXT,
    host_picture_url TEXT,
    host_neighbourhood TEXT,
    host_listings_count INT,
    host_total_listings_count INT,
    host_verifications TEXT[],
    host_has_profile_pic BOOLEAN,
    host_identity_verified BOOLEAN,
    neighbourhood TEXT,
    neighbourhood_cleansed TEXT,
    neighbourhood_group_cleansed TEXT,
    latitude DOUBLE,
    longitude DOUBLE,
    property_type TEXT,
    room_type TEXT,
    accommodates INT,
    bathrooms INT,
    bathrooms_text TEXT,
    bedrooms INT,
    beds INT,
    amenities TEXT[],
    price TEXT,
    minimum_nights INT,
    maximum_nights INT,
    minimum_minimum_nights INT,
    maximum_minimum_nights INT,
    minimum_maximum_nights INT,
    maximum_maximum_nights INT,
    minimum_nights_avg_ntm DOUBLE,
    maximum_nights_avg_ntm DOUBLE,
    calendar_updated TEXT,
    has_availability bool_sigla,
    availability_30 INT,
    availability_60 INT,
    availability_90 INT,
    availability_365 INT,
    calendar_last_scraped DATE,
    number_of_reviews INT,
    number_of_reviews_ltm INT,
    number_of_reviews_l30d INT,
    first_review DATE,
    last_review DATE,
    review_scores_rating DOUBLE,
    review_scores_accuracy DOUBLE,
    review_scores_cleanliness DOUBLE,
    review_scores_checkin DOUBLE,
    review_scores_communication DOUBLE,
    review_scores_location DOUBLE,
    review_scores_value DOUBLE,
    license TEXT,
    instant_bookable bool_sigla,
    calculated_host_listings_count INT,
    calculated_host_listings_count_entire_homes INT,
    calculated_host_listings_count_private_rooms INT,
    calculated_host_listings_count_shared_rooms INT,
    reviews_per_month DOUBLE
);

In [ ]:
DROP TABLE IF EXISTS Reviews CASCADE;
CREATE TABLE Reviews (
    id INT PRIMARY KEY,
    listing_id INT,
    date DATE,
    reviewer_id INT,
    reviewer_name TEXT,
    comments TEXT,
    FOREIGN KEY (listing_id) REFERENCES Listings(id)
);

In [ ]:
DROP TABLE IF EXISTS Calendar CASCADE;
CREATE TABLE Calendar (
    listing_id INT,
    date DATE,
    available BOOLEAN,
    price TEXT,
    adjusted_price TEXT,
    minimum_nights INT,
    maximum_nights INT,
    FOREIGN KEY (listing_id) REFERENCES Listings(id)
);